In [1]:

import numpy as np 
import pandas as pd


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/crop-recommendation-dataset/Crop_recommendation.csv


In [2]:
data = pd.read_csv("/kaggle/input/crop-recommendation-dataset/Crop_recommendation.csv")
data

,N,P,K,temperature,humidity,ph,rainfall,label
0,90,42,43,20.879744,82.002744,6.502985,202.935536,rice
1,85,58,41,21.770462,80.319644,7.038096,226.655537,rice
2,60,55,44,23.004459,82.320763,7.840207,263.964248,rice
3,74,35,40,26.491096,80.158363,6.980401,242.864034,rice
4,78,42,42,20.130175,81.604873,7.628473,262.717340,rice
...,...,...,...,...,...,...,...,...
2195,107,34,32,26.774637,66.413269,6.780064,177.774507,coffee
2196,99,15,27,27.417112,56.636362,6.086922,127.924610,coffee
2197,118,33,30,24.131797,67.225123,6.362608,173.322839,coffee
2198,117,32,34,26.272418,52.127394,6.758793,127.175293,coffee


In [3]:
x = data.drop('label', axis=1)
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
#mean variance standardaisation
x = sc.fit_transform(x)
y1 = data['label']
from sklearn.preprocessing import OneHotEncoder
categories=y1.unique()

#finding distance of datapoints from their cluster and assuming 3 points with lowest distance belongs to same cluster
on=[]
yld=[]
for i in range(len(x)):
    p_y=[]
    for j in range(22):
        clus=x[j*100:(j+1)*100]
        p_y.append(np.mean(np.linalg.norm((clus-x[i]), axis=1)))
    sm3=np.sort(p_y)[3]
    yld.append(1*(np.array(p_y)<sm3))

len(yld)

2200

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, yld, test_size = 0.1,
                                                    shuffle = True, random_state = 0)

In [6]:

from keras import backend as K

#custom metrics for evaluation of model
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [7]:
import keras
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(22, input_dim=7, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(12, activation='relu'))
#sigmoid activation with binary_crossentropy loss instead of 
# softmax with categorical ceossentropy because it is multi class multi lable dataset
model.add(Dense(22, activation='sigmoid'))


2023-02-11 03:25:47.158562: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [11]:
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=[f1_m,precision_m, recall_m])
history = model.fit(np.float32(np.array(X_train)), np.float32(np.array(y_train)), epochs=2000)

Epoch 1/2000
62/62 [==============================] - 1s 2ms/step - loss: 0.5012 - f1_m: 0.1353 - precision_m: 0.1960 - recall_m: 0.1108
Epoch 2/2000
62/62 [==============================] - 0s 2ms/step - loss: 0.3525 - f1_m: 0.0647 - precision_m: 0.4970 - recall_m: 0.0367
Epoch 3/2000
62/62 [==============================] - 0s 2ms/step - loss: 0.2725 - f1_m: 0.3356 - precision_m: 0.9393 - recall_m: 0.2106
Epoch 4/2000
62/62 [==============================] - 0s 2ms/step - loss: 0.2089 - f1_m: 0.5937 - precision_m: 0.9155 - recall_m: 0.4447
Epoch 5/2000
62/62 [==============================] - 0s 2ms/step - loss: 0.1624 - f1_m: 0.7255 - precision_m: 0.9078 - recall_m: 0.6070
Epoch 6/2000
62/62 [==============================] - 0s 2ms/step - loss: 0.1338 - f1_m: 0.7920 - precision_m: 0.9003 - recall_m: 0.7093
Epoch 7/2000
62/62 [==============================] - 0s 2ms/step - loss: 0.1161 - f1_m: 0.8284 - precision_m: 0.8972 - recall_m: 0.7714
Epoch 8/2000
62/62 [=====================

In [14]:
os.mkdir('/kaggle/working/modelf')
model.save('/kaggle/working/modelf')

In [32]:

x_hdkjkj=[[108],[22],[47],[28.5457],[91.727427],[6.161124],[25.129005]]
y_pred = model.predict(X_test)
print(np.argmax(y_pred[22]),np.argmax(y_test[22]))

pred = list()
for i in range(len(y_pred)):

    pred.append(1*(y_pred[i]>0.8))

test = list()
for i in range(len(y_test)):

    test.append(1*(y_test[i]>0.5))
print(pred[12],test[12])
from sklearn.metrics import accuracy_score ,f1_score,average_precision_score
a = accuracy_score(pred,test)
p = average_precision_score(pred,test)

print( a,p)

3 3
[0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0] [0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
0.7863636363636364 0.9176325573064237
